In [88]:
import numpy as np
from itertools import combinations
from copy import deepcopy

# dna sequence. make_kmers, dna_seq_kmers_random

In [60]:
dna_sequence= 'CTTTATTTTCACCACAAACAGAGATTAAAGAAAGTGTTGGATTAAAAGATGGTGTTAAAGAGTAAAAATTGACTTATTATACTCCTGAATACGAAACCAAAGATACTGATATCTTGGCAGCATTCCGAGTAACTCCTCAACCTGGAGTTCCACCCGAAGAAGCAGGGGCCGCGGTAGCTGCCGAATCTTCTACCGGTACATGGACAAGTGTGTGGACCGATGGACTTACCAGCCTTGATCGTTACAAAGGGCGATGCTACCACATCGAGCCCGTTCCTGGAGAAGAAGAGCAATATATTGCTTATGTAGCTTACCCCTTAGACCTTTTTGAAGAAGGTTCTGTTACTAACATGTTTACTTCCATTGTAGGTAATGTATTTGGGTTCAAAGCCCTACGCGCTCTACGTCTGGAAGATTTGCGAATCCCTCCGGCTTATATTAAAACCTTCCAAGGCCCGCCTCATGGCATCCAGGTTGAGAGAGATAAATTGAACAAATATGGTCGTCCCCTGTTGGGATGTACTATTAAACCTAAATTGGGGTTATCCGCTAAAAACTACGGTAGGGCATGTTATGAATGTCTTCGTGGTGGACTTGATTTTACCAAAGATGATGAAAACGTGAACTCCCAACCGTTTATGCGTTGGAGAGATCGATTCTTATTTGTAGCAAAAA'

len(dna_sequence)

675

In [108]:
def kmers(seq,k):
        kmer_dic={}
        kmers_list = []
        for i in range(len(seq)-(k-1)):
                k_mer=seq[i:i+k]
                kmers_list.append(k_mer)
                if k_mer in kmer_dic:
                        kmer_dic[k_mer]+=1
                else:
                        kmer_dic[k_mer]=1
        return kmers_list

In [124]:
dna_kmers = kmers(dna_sequence, 670)
dna_kmers

['CTTTATTTTCACCACAAACAGAGATTAAAGAAAGTGTTGGATTAAAAGATGGTGTTAAAGAGTAAAAATTGACTTATTATACTCCTGAATACGAAACCAAAGATACTGATATCTTGGCAGCATTCCGAGTAACTCCTCAACCTGGAGTTCCACCCGAAGAAGCAGGGGCCGCGGTAGCTGCCGAATCTTCTACCGGTACATGGACAAGTGTGTGGACCGATGGACTTACCAGCCTTGATCGTTACAAAGGGCGATGCTACCACATCGAGCCCGTTCCTGGAGAAGAAGAGCAATATATTGCTTATGTAGCTTACCCCTTAGACCTTTTTGAAGAAGGTTCTGTTACTAACATGTTTACTTCCATTGTAGGTAATGTATTTGGGTTCAAAGCCCTACGCGCTCTACGTCTGGAAGATTTGCGAATCCCTCCGGCTTATATTAAAACCTTCCAAGGCCCGCCTCATGGCATCCAGGTTGAGAGAGATAAATTGAACAAATATGGTCGTCCCCTGTTGGGATGTACTATTAAACCTAAATTGGGGTTATCCGCTAAAAACTACGGTAGGGCATGTTATGAATGTCTTCGTGGTGGACTTGATTTTACCAAAGATGATGAAAACGTGAACTCCCAACCGTTTATGCGTTGGAGAGATCGATTCTTATTTGTAGC',
 'TTTATTTTCACCACAAACAGAGATTAAAGAAAGTGTTGGATTAAAAGATGGTGTTAAAGAGTAAAAATTGACTTATTATACTCCTGAATACGAAACCAAAGATACTGATATCTTGGCAGCATTCCGAGTAACTCCTCAACCTGGAGTTCCACCCGAAGAAGCAGGGGCCGCGGTAGCTGCCGAATCTTCTACCGGTACATGGACAAGTGTGTGGACCGATGGACTTACCAGCCTTGATCGTTACAAAGGGCGATGCTACCACATCGAGCCCGTTCCTGGAGAAGAAGAGCAATATATTGCTTATGTAGCTTACCCCTTAGACC

In [125]:
len(dna_kmers)

6

In [79]:
dna_kmers_random = [dna_kmers[0], dna_kmers[15], dna_kmers[25], dna_kmers[45], dna_kmers[58],
                    dna_kmers[68], dna_kmers[80], dna_kmers[105], dna_kmers[135], dna_kmers[150]]

dna_kmers_random

['CTTTATTTTCACCACAAACAGAGATTAAAGAAAGTGTTGGATTAAAAGAT',
 'AAACAGAGATTAAAGAAAGTGTTGGATTAAAAGATGGTGTTAAAGAGTAA',
 'TAAAGAAAGTGTTGGATTAAAAGATGGTGTTAAAGAGTAAAAATTGACTT',
 'AAGATGGTGTTAAAGAGTAAAAATTGACTTATTATACTCCTGAATACGAA',
 'AGAGTAAAAATTGACTTATTATACTCCTGAATACGAAACCAAAGATACTG',
 'TTGACTTATTATACTCCTGAATACGAAACCAAAGATACTGATATCTTGGC',
 'ACTCCTGAATACGAAACCAAAGATACTGATATCTTGGCAGCATTCCGAGT',
 'CTGATATCTTGGCAGCATTCCGAGTAACTCCTCAACCTGGAGTTCCACCC',
 'CTCAACCTGGAGTTCCACCCGAAGAAGCAGGGGCCGCGGTAGCTGCCGAA',
 'CACCCGAAGAAGCAGGGGCCGCGGTAGCTGCCGAATCTTCTACCGGTACA']

# pairwise alignment code
(only SW)

In [64]:
def read_blosum(file):
    with open(file, 'r') as f:
        matrix = f.readlines()
    del matrix[-1]
    score_dic = {}  # dictionary to store scores of all the pairs
    cols = matrix[0].split()
    for line in range(1, len(matrix)):
        entries = matrix[line].split()
        row = entries[0]  # nucleotide
        scores = entries[1:]  # scores
        for score, col in zip(scores, cols):
            key = ''.join(sorted((row, col)))
            score_dic[key] = int(score)
    return score_dic


def pairwise_alignment(seq1, seq2, w, scoring, print_output=False):
    """Smith-Waterman local alignment"""
    n = len(seq1)
    m = len(seq2)

    # INITIALIZE MATRIX
    mat = np.zeros((n + 1, m + 1), dtype=int)
    for j in range(m + 1):  # first row
        mat[0, j] = j * w
    for i in range(n + 1):  # first column
        mat[i, 0] = i * w

    # FILL MATRIX
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            pair = ''.join(sorted((seq1[i - 1], seq2[j - 1])))
            diag = mat[i - 1, j - 1] + scoring[pair]
            up = mat[i - 1, j] + w
            left = mat[i, j - 1] + w
            mat[i, j] = max(diag, up, left, 0)  # replacing all negative values with zero

    alignment_score = np.amax(mat)

    # TRACEBACK ALIGNMENT
    alseq1 = ''
    alseq2 = ''  # storing the gaps in alignment

    max_index = np.where(mat == alignment_score)
    i, j = max_index[0][0], max_index[1][0]  # starting from max value
    while mat[i, j] != 0:
        key = ''.join(sorted((seq1[i - 1], seq2[j - 1])))
        diag_score = mat[i - 1, j - 1] + scoring[key]
        up_score = mat[i - 1, j] + w
        left_score = mat[i, j - 1] + w
        # finding the direction of pointer from which the score was taken
        if mat[i, j] == diag_score:
            alseq1 += seq1[i - 1]
            alseq2 += seq2[j - 1]
            i -= 1
            j -= 1
        elif mat[i, j] == up_score:
            alseq1 += seq1[i - 1]
            alseq2 += '-'
            i -= 1
        elif mat[i, j] == left_score:
            alseq1 += '-'
            alseq2 += seq2[j - 1]
            j -= 1
        else:
            raise Exception('This should never happen')

    # correct order of string since we traced backwards
    aligned_seq1 = alseq1[::-1]
    aligned_seq2 = alseq2[::-1]

    # for matching base pair at the start of a sequence
    # for a match preceding the alignment at start of any one sequence
    pos1 = seq1.find(aligned_seq1) - 1
    pos2 = seq2.find(aligned_seq2) - 1

    if seq1[pos1] == seq2[pos2]:
        aligned_seq1 = seq1[pos1] + aligned_seq1
        aligned_seq2 = seq2[pos2] + aligned_seq2
        alignment_score += 1

    # Alignment statistics
    matches = ''
    id_count = 0  # sequence identity count
    ss_count = 0  # sequence similarity count
    for aa1, aa2 in zip(aligned_seq1, aligned_seq2):
        pair = ''.join(sorted((aa1, aa2)))
        if aa1 == '-' or aa2 == '-':
            matches += ' '
        elif aa1 == aa2:
            id_count += 1
            matches += '|'
        elif scoring[pair] >= 0:
            ss_count += 1
            matches += ':'
        else:
            matches += ' '

    seq_identity = round(id_count / len(aligned_seq1) * 100, 2)
    seq_similarity = round(ss_count / len(aligned_seq1) * 100, 2)

    # FORMATTED OUTPUT
    if print_output is True:
        print(f'Alignment Score = {alignment_score}')
        print(f'Sequence Identity = {seq_identity}%')
        print(f'Sequence Similarity = {seq_similarity}%')

        for i in range(0, len(aligned_seq1) - 1, 80):
            print(aligned_seq1[i: i + 80])
            print(matches[i: i + 80])
            print(aligned_seq2[i: i + 80])
            print()

    return alignment_score, seq_identity, aligned_seq1


In [65]:
dna_scores = 'C:\\Users\\kriti\\Desktop\\SEM_3\\PLAB_II\\Group_Project\\Group_4_env\\Amin-plab-4_solution2\\dna_scores.txt'
d_scores = read_blosum(dna_scores)
d_scores

{'AA': 1,
 'AT': -4,
 'AG': -4,
 'AC': -4,
 'TT': 1,
 'GT': -4,
 'CT': -4,
 'GG': 1,
 'CG': -4,
 'CC': 1}

In [103]:
pairwise_alignment(seq1=dna_kmers_random[0], seq2=dna_kmers_random[1], w=-4, scoring=d_scores, print_output=True)

Alignment Score = 35
Sequence Identity = 100.0%
Sequence Similarity = 0.0%
AAACAGAGATTAAAGAAAGTGTTGGATTAAAAGAT
|||||||||||||||||||||||||||||||||||
AAACAGAGATTAAAGAAAGTGTTGGATTAAAAGAT



(35, 100.0, 'AAACAGAGATTAAAGAAAGTGTTGGATTAAAAGAT')

# perform MSA and reconstruct the genomic sequence

In [67]:
def find_overlapping_ends(seq1, seq2, aligned_seq):
    """Find if aligned sequence is present at ends of both sequence.
    Returns false if present in middle or if not present."""
    if seq1.startswith(aligned_seq) and seq2.endswith(aligned_seq):
        return True
    elif seq2.startswith(aligned_seq) and seq1.endswith(aligned_seq):
        return True
    else:
        return False

def merge_overlapping_ends(seq1, seq2, aligned_seq):
    """aligned seq should be present in both and ovelap at ends"""
    ind1, ind2 = seq1.find(aligned_seq), seq2.find(aligned_seq)
    assert ind1 != -1
    assert ind2 != -1
    if ind1 > ind2:
        return seq1[:ind1]+seq2
    elif ind2 > ind1:
        return seq2[:ind2]+seq1

In [104]:
def perform_msa(sequences, score_mat):
    all_pairs = [(i,j) for i, j in combinations(sequences, 2)]
    pairwise_scores = {}
    # perform pairwise alignment of all sequences
    for seq1, seq2 in all_pairs:
        score, seq_id, alignment = pairwise_alignment(seq1=seq1, seq2=seq2, w=-4, scoring=score_mat)
        pairwise_scores[(seq1, seq2)] = (seq_id, score, alignment)

    return pairwise_scores


def seq_assembly(sequences, score_mat):
    # all sequences in multiple sequence alignment
    # all_seqs = set()
    # for s1, s2 in pairwise_scores.keys():
    #     all_seqs.update([s1, s2])

    seqs = deepcopy(sequences)
    while len(seqs) > 1:
        # perform msa
        pairwise_scores = perform_msa(sequences=seqs, score_mat=score_mat)
        # sort pairwise alignments based on scores
        sorted_scores = sorted(pairwise_scores.items(), key=lambda x: x[1], reverse=True)
        for scores in sorted_scores:
            seq1, seq2 = scores[0]
            alignment = scores[1][2]

            # find overlapping ends
            if find_overlapping_ends(seq1=seq1, seq2=seq2, aligned_seq=alignment):
                merged_seq = merge_overlapping_ends(seq1=seq1, seq2=seq2, aligned_seq=alignment)
                seqs.remove(seq1)
                seqs.remove(seq2)
                seqs.append(merged_seq)
                break
            else:  # if no overlapping ends, check next highest scoring pairwise alignment
                pass
    return seqs[0]



In [82]:
dna_kmers_random

['CTTTATTTTCACCACAAACAGAGATTAAAGAAAGTGTTGGATTAAAAGAT',
 'AAACAGAGATTAAAGAAAGTGTTGGATTAAAAGATGGTGTTAAAGAGTAA',
 'TAAAGAAAGTGTTGGATTAAAAGATGGTGTTAAAGAGTAAAAATTGACTT',
 'AAGATGGTGTTAAAGAGTAAAAATTGACTTATTATACTCCTGAATACGAA',
 'AGAGTAAAAATTGACTTATTATACTCCTGAATACGAAACCAAAGATACTG',
 'TTGACTTATTATACTCCTGAATACGAAACCAAAGATACTGATATCTTGGC',
 'ACTCCTGAATACGAAACCAAAGATACTGATATCTTGGCAGCATTCCGAGT',
 'CTGATATCTTGGCAGCATTCCGAGTAACTCCTCAACCTGGAGTTCCACCC',
 'CTCAACCTGGAGTTCCACCCGAAGAAGCAGGGGCCGCGGTAGCTGCCGAA',
 'CACCCGAAGAAGCAGGGGCCGCGGTAGCTGCCGAATCTTCTACCGGTACA']

In [106]:
seq_assembly(sequences=dna_kmers_random, score_mat=d_scores)

'CTTTATTTTCACCACAAACAGAGATTAAAGAAAGTGTTGGATTAAAAGATGGTGTTAAAGAGTAAAAATTGACTTATTATACTCCTGAATACGAAACCAAAGATACTGATATCTTGGCAGCATTCCGAGTAACTCCTCAACCTGGAGTTCCACCCGAAGAAGCAGGGGCCGCGGTAGCTGCCGAATCTTCTACCGGTACA'

In [107]:
dna_kmers_random

['CTTTATTTTCACCACAAACAGAGATTAAAGAAAGTGTTGGATTAAAAGAT',
 'AAACAGAGATTAAAGAAAGTGTTGGATTAAAAGATGGTGTTAAAGAGTAA',
 'TAAAGAAAGTGTTGGATTAAAAGATGGTGTTAAAGAGTAAAAATTGACTT',
 'AAGATGGTGTTAAAGAGTAAAAATTGACTTATTATACTCCTGAATACGAA',
 'AGAGTAAAAATTGACTTATTATACTCCTGAATACGAAACCAAAGATACTG',
 'TTGACTTATTATACTCCTGAATACGAAACCAAAGATACTGATATCTTGGC',
 'ACTCCTGAATACGAAACCAAAGATACTGATATCTTGGCAGCATTCCGAGT',
 'CTGATATCTTGGCAGCATTCCGAGTAACTCCTCAACCTGGAGTTCCACCC',
 'CTCAACCTGGAGTTCCACCCGAAGAAGCAGGGGCCGCGGTAGCTGCCGAA',
 'CACCCGAAGAAGCAGGGGCCGCGGTAGCTGCCGAATCTTCTACCGGTACA']

In [126]:
predicted = seq_assembly(sequences=dna_kmers, score_mat=d_scores)
predicted

'CTTTATTTTCACCACAAACAGAGATTAAAGAAAGTGTTGGATTAAAAGATGGTGTTAAAGAGTAAAAATTGACTTATTATACTCCTGAATACGAAACCAAAGATACTGATATCTTGGCAGCATTCCGAGTAACTCCTCAACCTGGAGTTCCACCCGAAGAAGCAGGGGCCGCGGTAGCTGCCGAATCTTCTACCGGTACATGGACAAGTGTGTGGACCGATGGACTTACCAGCCTTGATCGTTACAAAGGGCGATGCTACCACATCGAGCCCGTTCCTGGAGAAGAAGAGCAATATATTGCTTATGTAGCTTACCCCTTAGACCTTTTTGAAGAAGGTTCTGTTACTAACATGTTTACTTCCATTGTAGGTAATGTATTTGGGTTCAAAGCCCTACGCGCTCTACGTCTGGAAGATTTGCGAATCCCTCCGGCTTATATTAAAACCTTCCAAGGCCCGCCTCATGGCATCCAGGTTGAGAGAGATAAATTGAACAAATATGGTCGTCCCCTGTTGGGATGTACTATTAAACCTAAATTGGGGTTATCCGCTAAAAACTACGGTAGGGCATGTTATGAATGTCTTCGTGGTGGACTTGATTTTACCAAAGATGATGAAAACGTGAACTCCCAACCGTTTATGCGTTGGAGAGATCGATTCTTATTTGTAGCAAAAA'

In [127]:
predicted == dna_sequence

True

In [138]:
dna_scores = 'C:\\Users\\kriti\\Desktop\\SEM_3\\PLAB_II\\Group_Project\\Group_4_env\\Amin-plab-4_solution2\\dna_scores.txt'
d_scores = read_blosum(dna_scores)
d_scores

{'AA': 1,
 'AT': -4,
 'AG': -4,
 'AC': -4,
 'TT': 1,
 'GT': -4,
 'CT': -4,
 'GG': 1,
 'CG': -4,
 'CC': 1}

# extra functions

- read fasta file
- create file with assembled sequence
- create file with list of predicted ammino acids
- create file with list of predicted proteins
- create function for checking if list sequences is of nucleotide or amino acids.

In [3]:
fastq = 'C:\\Users\\kriti\\Desktop\\SEM_3\\PLAB_II\\Group_Project\\Group_4_env\\fastq_dummy.fastq'

In [4]:
with open(fastq, 'r') as fifi:
    fasq = fifi.read()

fasq

'@MN00537:51:000H2K25G:1:11101:2213:1092 1:N:0:9\nCTCCAGTCCTTACTCCCATATCTAACCTCTT\n+\nFFFFFFFFFFFFAFFFFFFFF=FFFFAFFFFFFF/AFFF#FFFFFFFFF#FFFFFFFF\n\n@MN00675:51:000H2K25G:1:11101:2213:1092 1:N:0:9\nCTCCAGTCCTTACTCCGGGGGGGGCATATCT\n+\nFFFFFFFFFFFFAFFFFFFFF=FFFFFFFFFFFF/AFFF#FFFFFFFFF#FFFFFFFF\n\n@MN00747:51:000H2K25G:1:11101:2213:1092 1:N:0:9\nCTCCAGTCCTTACTCCCATATCTAACCTAAA\n+\nFFFFFFFFFFFFAFFFFFFFF=FFFFAFFFFFFF/AFFF#FFFFFFFFF#FFFFFFFF\n\n@MN00457:51:000H2K25G:1:11101:2213:1092 1:N:0:9\nCTCCAGTCCTTACCCCCCCCCCCTCCCATAT\n+\nFFFFFFFFFFFFAFFFFFFFF=FFFFAFFFFFFF/AFFF#FFFFFFFFF#FFFFFFFF'

In [8]:
%run sequence_assembly.py

start
fasta
['AAAAGAGTCTAAAGGTTAAGAT', 'GGTTAAGATGATCAATTAACAA']
AAAAGAGTCTAAAGGTTAAGATGATCAATTAACAA
fastq
['CCCATATCTAGGGCA', 'TAGGGCATATTTACT', 'ATATCTAGGGCATAT', 'TTTACTCCCGTATCA']
CCCATATCTAGGGCATATTTACTCCCGTATCA
end


In [11]:
!coverage run -m pytest sequence_assembly.py

============================= test session starts =============================
platform win32 -- Python 3.9.9, pytest-6.2.5, py-1.11.0, pluggy-1.0.0
rootdir: C:\Users\kriti\Desktop\PLAB2_venv\group_4\backend_code
collected 0 items

============================ no tests ran in 0.27s ============================


In [ ]:

if __name__ == '__main__':
    print('start')
    print('fasta')
    obj =Assembly(sequences='fasta_dummy.fasta')
    print(obj.seqs)
    print(obj.assembled_sequence)
    print('fastq')
    obj2 = Assembly(sequences='fastq_dummy.fastq')
    print(obj2.seqs)
    print(obj2.assembled_sequence)
    print('end')